#  Oral Cancer Detection using AI

##  Installation and Setup


In [ ]:
%pip install ultralytics
%pip install albumentations
%pip install opencv-python
%pip install scikit-learn
%pip install seaborn
%pip install plotly
%pip install tqdm
%pip install tensorboard
%pip install pycocotools

import torch
import torchvision
import ultralytics
import cv2
import albumentations as A
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
import warnings
warnings.filterwarnings('ignore')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.5 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


##  Google Drive Setup

In [3]:
from google.colab import drive
drive.mount('/content/drive')

DATA_ROOT = "/content/drive/MyDrive/ORAL"
ANNOTATION_FILE = os.path.join(DATA_ROOT, "Annotation.json")
MODIFIED_ANNOTATION_FILE = os.path.join(DATA_ROOT, "Modified_Annotation.json")
TRAIN_DIR = os.path.join(DATA_ROOT, "Training")
VAL_DIR = os.path.join(DATA_ROOT, "Validation")
OUTPUT_DIR = "/content/outputs"

os.makedirs(OUTPUT_DIR, exist_ok=True)

CLASS_MAPPING = {
    'Healthy': 0,
    'Benign': 1,
    'OPMD': 2,
    'OCA': 3
}

REVERSE_CLASS_MAPPING = {v: k for k, v in CLASS_MAPPING.items()}
CLASS_NAMES = list(CLASS_MAPPING.keys())


Mounted at /content/drive


## Modify Annotation File

Modifying the annotation file as:
1. Single Oral Cavity box → Healthy
2. Multiple boxes → class determined by folder location
3. Remove images not available in any folders

In [4]:

if not os.path.exists(MODIFIED_ANNOTATION_FILE):
    raise FileNotFoundError(f" Modified annotation file not found: {MODIFIED_ANNOTATION_FILE}. Please run the annotation modification script first.")

with open(MODIFIED_ANNOTATION_FILE, 'r') as f:
    modified_annotations = json.load(f)

print(f"\n Categories in modified file:")
for cat in modified_annotations['categories']:
    print(f"  {cat['id']}: {cat['name']}")



 Categories in modified file:
  0: Healthy
  1: Benign
  2: OPMD
  3: OCA


##  YOLO Model Architecture


In [5]:
import shutil
import yaml

def load_annotations(annotation_path):
    with open(annotation_path, 'r') as f:
        annotations = json.load(f)
    return annotations


def prepare_yolo_dataset(annotations, data_root, output_dir):

    yolo_dir = os.path.join(output_dir, 'yolo_dataset')
    os.makedirs(yolo_dir, exist_ok=True)

    for split in ['train', 'val']:
        os.makedirs(os.path.join(yolo_dir, split, 'images'), exist_ok=True)
        os.makedirs(os.path.join(yolo_dir, split, 'labels'), exist_ok=True)

    image_map = {img['file_name']: img for img in annotations['images']}

    def get_image_dimensions(image_path):
        try:
            img = cv2.imread(image_path)
            if img is not None:
                height, width = img.shape[:2]
                return width, height
            else:
                print(f"Warning: Could not load image {image_path}")
                return None, None
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")
            return None, None

    train_annotations = []
    train_split_path = os.path.join(data_root, 'Training')

    for class_name in CLASS_NAMES:
        class_path = os.path.join(train_split_path, class_name)
        if os.path.exists(class_path):
            for filename in os.listdir(class_path):
                if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                    if filename in image_map:
                        img_info = image_map[filename]

                        src_path = os.path.join(class_path, filename)
                        dst_path = os.path.join(yolo_dir, 'train', 'images', filename)
                        shutil.copy2(src_path, dst_path)

                        img_w, img_h = get_image_dimensions(src_path)
                        if img_w is None or img_h is None:
                            continue

                        img_annotations = [ann for ann in annotations['annotations'] if ann['image_id'] == img_info['id']]

                        label_path = os.path.join(yolo_dir, 'train', 'labels',
                                                filename.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt'))

                        with open(label_path, 'w') as f:
                            for ann in img_annotations:
                                x, y, w, h = ann['bbox']

                                x_center = (x + w/2) / img_w
                                y_center = (y + h/2) / img_h
                                width = w / img_w
                                height = h / img_h

                                class_id = ann['category_id']

                                f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

                        train_annotations.append({
                            'image_id': img_info['id'],
                            'filename': filename,
                            'annotations': len(img_annotations)
                        })

    val_annotations = []
    val_split_path = os.path.join(data_root, 'Validation')

    for class_name in CLASS_NAMES:
        class_path = os.path.join(val_split_path, class_name)
        if os.path.exists(class_path):
            for filename in os.listdir(class_path):
                if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                    if filename in image_map:
                        img_info = image_map[filename]

                        src_path = os.path.join(class_path, filename)
                        dst_path = os.path.join(yolo_dir, 'val', 'images', filename)
                        shutil.copy2(src_path, dst_path)

                        img_w, img_h = get_image_dimensions(src_path)
                        if img_w is None or img_h is None:
                            continue

                        img_annotations = [ann for ann in annotations['annotations'] if ann['image_id'] == img_info['id']]

                        label_path = os.path.join(yolo_dir, 'val', 'labels',
                                                filename.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt'))

                        with open(label_path, 'w') as f:
                            for ann in img_annotations:
                                x, y, w, h = ann['bbox']

                                x_center = (x + w/2) / img_w
                                y_center = (y + h/2) / img_h
                                width = w / img_w
                                height = h / img_h

                                class_id = ann['category_id']

                                f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

                        val_annotations.append({
                            'image_id': img_info['id'],
                            'filename': filename,
                            'annotations': len(img_annotations)
                        })

    dataset_yaml = {
        'path': yolo_dir,
        'train': 'train/images',
        'val': 'val/images',
        'nc': 4,
        'names': CLASS_NAMES
    }

    yaml_path = os.path.join(yolo_dir, 'dataset.yaml')
    with open(yaml_path, 'w') as f:
        yaml.dump(dataset_yaml, f, default_flow_style=False)

    return yolo_dir, yaml_path

annotations = load_annotations(MODIFIED_ANNOTATION_FILE)
yolo_dir, yaml_path = prepare_yolo_dataset(annotations, DATA_ROOT, OUTPUT_DIR)


##  YOLO Training

In [6]:
from ultralytics import YOLO
model = YOLO('yolov8s.pt')

results = model.train(
    data=yaml_path,
    epochs=200,
    imgsz=640,
    batch=32,
    device=0 if torch.cuda.is_available() else 'cpu',
    project=OUTPUT_DIR,
    name='yolo_oral_cancer_detection',
    save=True,
    save_period=10,
    cache=True,
    workers=4,
    patience=30,
    lr0=0.01,
    lrf=0.1,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3,
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,
    label_smoothing=0.05,
    nbs=64,
    val=True,
    plots=True,
    verbose=True
)

best_model_path = os.path.join(OUTPUT_DIR, 'yolo_oral_cancer_detection', 'weights', 'best.pt')
if os.path.exists(best_model_path):
    best_model = YOLO(best_model_path)
    print(f"Best model loaded from: {best_model_path}")
else:
    best_model = model


WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in in the future.
Ultralytics 8.3.195 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/outputs/yolo_dataset/dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.1, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo_oral_cancer_detection, nbs=64, nms=False, opset=None, opt

##  Model Evaluation and Testing

In [7]:
yolo_results = best_model.val(data=yaml_path)

print(f"\n YOLO Results:")
print(f"mAP50: {yolo_results.box.map50:.4f}")
print(f"mAP50-95: {yolo_results.box.map:.4f}")
print(f"Precision: {yolo_results.box.mp:.4f}")
print(f"Recall: {yolo_results.box.mr:.4f}")

print(f"\n Per-Class Results:")
for i, class_name in enumerate(CLASS_NAMES):
    if hasattr(yolo_results.box, 'maps') and len(yolo_results.box.maps) > i:
        print(f"{class_name}: mAP50 = {yolo_results.box.maps[i]:.4f}")

evaluation_results = {
    'yolo_detection': {
        'map50': float(yolo_results.box.map50),
        'map50_95': float(yolo_results.box.map),
        'precision': float(yolo_results.box.mp),
        'recall': float(yolo_results.box.mr),
        'classes': CLASS_NAMES
    }
}

with open(os.path.join(OUTPUT_DIR, 'evaluation_results.json'), 'w') as f:
    json.dump(evaluation_results, f, indent=2)

print(f" Evaluation results saved to {OUTPUT_DIR}/evaluation_results.json")


Ultralytics 8.3.195 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,127,132 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1713.6±1810.8 MB/s, size: 5430.4 KB)
val: Scanning /content/outputs/yolo_dataset/val/labels.cache... 300 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 300/300 383.4Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 19/19 0.5it/s 36.7s
                   all        300        666       0.46      0.317      0.341       0.29
               Healthy         62         62      0.619      0.484      0.604      0.558
                Benign         78        205      0.362      0.302      0.267      0.213
                  OPMD        150        379      0.551      0.383      0.381       0.29
                   OCA         10         20       0.31        0.1      0.112      0.101
Speed: 0.2ms preprocess, 7.9ms 